In [ ]:
# Quantum Simulation Analysis Tools Demo

"""
Demonstration of the analysis module capabilities.
This notebook shows how to use the analysis tools for:
- Quantum state analysis and comparison
- Error metrics calculation
- Spectrum analysis (FFT, absorption spectra)
- Performance benchmarking
- Data export
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from quantum_simulation import (
    create_quantum_system,
    create_exact_solver,
    create_trotter_solver,
    create_magnus_solver,
    create_state_analyzer,
    create_spectrum_analyzer,
    create_performance_analyzer,
    DataExporter,
    SimulationConfig
)

print("=== Quantum Simulation Analysis Demo ===")

# Setup configuration for analysis demo
analysis_config = SimulationConfig(
    time_end=50.0,       # Reasonable time range
    num_time_points=20,  # Enough points for analysis
    max_optimization_steps=50,  # Quick VQA for demo
    num_layers=2         # Simple ansatz
)

print(f"Analysis configuration:")
print(f"- Time range: {analysis_config.time_start} to {analysis_config.time_end}")
print(f"- Time points: {analysis_config.num_time_points}")
print(f"- Molecule: {analysis_config.atom_string}")


In [ ]:
# Step 1: Setup system and run multiple solvers

print("\n=== Step 1: Running Multiple Solvers ===")

# Create quantum system
system = create_quantum_system(analysis_config)
ground_state, ground_energy = system.compute_ground_state_vqe()
time_points = analysis_config.get_time_points()

print(f"System created with {system.static_hamiltonian.num_qubits} qubits")
print(f"Ground state energy: {ground_energy:.6f} hartree")

# Initialize performance analyzer for benchmarking
perf_analyzer = create_performance_analyzer()

# Run multiple solvers and collect results
results_dict = {}

# 1. Exact solver
print("\nRunning Exact ODE solver...")
exact_solver = create_exact_solver(system, analysis_config)
perf_data_exact = perf_analyzer.benchmark_solver(
    exact_solver.evolve, "Exact", time_points, ground_state
)
results_dict['exact'] = perf_data_exact

# 2. Trotter 1st order
print("Running Trotter 1st order...")
trotter1_solver = create_trotter_solver(system, analysis_config, order=1)
perf_data_trotter1 = perf_analyzer.benchmark_solver(
    trotter1_solver.evolve, "Trotter1", time_points, ground_state
)
results_dict['trotter1'] = perf_data_trotter1

# 3. Trotter 2nd order
print("Running Trotter 2nd order...")
trotter2_solver = create_trotter_solver(system, analysis_config, order=2)
perf_data_trotter2 = perf_analyzer.benchmark_solver(
    trotter2_solver.evolve, "Trotter2", time_points, ground_state
)
results_dict['trotter2'] = perf_data_trotter2

# 4. Magnus 2nd order
print("Running Magnus 2nd order...")
magnus_solver = create_magnus_solver(system, analysis_config)
perf_data_magnus = perf_analyzer.benchmark_solver(
    magnus_solver.evolve, "Magnus2", time_points, ground_state
)
results_dict['magnus'] = perf_data_magnus

print(f"\nCompleted {len(results_dict)} solver runs")
print("Solver success status:")
for method, data in results_dict.items():
    status = "✅ Success" if data['success'] else "❌ Failed"
    time_taken = data['wall_time'] if data['success'] else 0
    print(f"  {method:10}: {status:10} ({time_taken:.3f}s)")

# Store actual evolved states for analysis
evolved_states_dict = {}
if results_dict['exact']['success']:
    evolved_states_dict['exact'] = exact_solver.evolve(time_points, ground_state)
if results_dict['trotter1']['success']:
    evolved_states_dict['trotter1'] = trotter1_solver.evolve(time_points, ground_state)
if results_dict['trotter2']['success']:
    evolved_states_dict['trotter2'] = trotter2_solver.evolve(time_points, ground_state)
if results_dict['magnus']['success']:
    evolved_states_dict['magnus'] = magnus_solver.evolve(time_points, ground_state)


In [ ]:
# Step 2: Quantum State Analysis and Method Comparison

print("\n=== Step 2: Quantum State Analysis ===")

# Create state analyzer
state_analyzer = create_state_analyzer(system, analysis_config)

# Perform comprehensive method comparison
print("Performing method comparison analysis...")
comparison_results = state_analyzer.compare_methods(evolved_states_dict, reference_method='exact')

print(f"Analysis completed:")
print(f"- Reference method: {comparison_results.metadata['reference_method']}")
print(f"- Methods compared: {comparison_results.metadata['num_methods']}")
print(f"- Time points: {comparison_results.metadata['num_time_points']}")
print(f"- Time range: {comparison_results.metadata['time_range']}")

# Display dipole moment error metrics
print("\nDipole Moment Error Metrics (vs Exact):")
print("-" * 50)
dipole_errors = comparison_results.data['dipole_errors']
for method, errors in dipole_errors.items():
    print(f"{method:12}: MAE={errors['mae']:.2e}, Max={errors['max_error']:.2e}, RMSE={errors['rmse']:.2e}")

# Display average fidelities
print("\nAverage State Fidelities (vs Exact):")
print("-" * 40)
fidelities = comparison_results.data['fidelities']
for method, fid_array in fidelities.items():
    if method != 'exact':
        avg_fidelity = np.mean(fid_array[~np.isnan(fid_array)])
        min_fidelity = np.min(fid_array[~np.isnan(fid_array)])
        print(f"{method:12}: Avg={avg_fidelity:.6f}, Min={min_fidelity:.6f}")

# Quick plot of dipole moments
plt.figure(figsize=(10, 6))
dipole_data = comparison_results.data['dipole_moments']
times = comparison_results.data['times']

for method, dipole_vals in dipole_data.items():
    plt.plot(times, dipole_vals, label=method, marker='o', markersize=4)

plt.xlabel('Time (a.u.)')
plt.ylabel('Dipole moment (a.u.)')
plt.title('Dipole Moment Evolution Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("✅ State analysis completed successfully!")


In [ ]:
# Step 3: Spectrum Analysis and Absorption Spectra

print("\n=== Step 3: Spectrum Analysis ===")

# Create spectrum analyzer
spectrum_analyzer = create_spectrum_analyzer(system, analysis_config)

# Analyze absorption spectra for different methods
print("Computing absorption spectra...")
spectra_results = {}

for method, dipole_vals in comparison_results.data['dipole_moments'].items():
    print(f"  Computing spectrum for {method}...")
    try:
        spectrum_data = spectrum_analyzer.analyze_absorption_spectrum(
            times=np.array(times),
            dipole_moments=dipole_vals,
            damping_gamma=0.001,
            window_type='blackman',
            normalize=True
        )
        spectra_results[method] = spectrum_data
        print(f"    ✅ {method}: {len(spectrum_data['omega_ev'])} frequency points")
    except Exception as e:
        print(f"    ❌ {method}: Failed - {e}")

# Plot absorption spectra comparison
plt.figure(figsize=(12, 8))

# Plot 1: All spectra
plt.subplot(2, 1, 1)
for method, spectrum_data in spectra_results.items():
    omega_ev = spectrum_data['omega_ev']
    sigma = spectrum_data['absorption_spectrum']
    plt.plot(omega_ev, sigma, label=f'{method}', linewidth=2)

plt.xlabel('Energy (eV)')
plt.ylabel('Normalized absorption')
plt.title('Absorption Spectra Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 2)  # Focus on low energy region

# Plot 2: Zoom into main peak region
plt.subplot(2, 1, 2)
for method, spectrum_data in spectra_results.items():
    omega_ev = spectrum_data['omega_ev']
    sigma = spectrum_data['absorption_spectrum']
    mask = (omega_ev >= 0) & (omega_ev <= 1)  # Zoom region
    plt.plot(omega_ev[mask], sigma[mask], label=f'{method}', linewidth=2, marker='o', markersize=3)

plt.xlabel('Energy (eV)')
plt.ylabel('Normalized absorption')
plt.title('Absorption Spectra (Zoomed)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Spectrum analysis summary
print("\nSpectrum Analysis Summary:")
print("-" * 40)
for method, spectrum_data in spectra_results.items():
    omega_ev = spectrum_data['omega_ev']
    sigma = spectrum_data['absorption_spectrum']
    
    # Find peak position
    peak_idx = np.argmax(sigma)
    peak_energy = omega_ev[peak_idx]
    peak_intensity = sigma[peak_idx]
    
    print(f"{method:12}: Peak at {peak_energy:.3f} eV (intensity: {peak_intensity:.3f})")

print("✅ Spectrum analysis completed successfully!")


In [ ]:
# Step 4: Performance Analysis and Benchmarking

print("\n=== Step 4: Performance Analysis ===")

# Generate comprehensive performance report
print("Generating performance report...")
performance_report = perf_analyzer.generate_performance_report(comparison_results)

# Display performance summary
print("\nPerformance Summary:")
print("=" * 60)
perf_df = performance_report['performance_summary']
if not perf_df.empty:
    for _, row in perf_df.iterrows():
        solver_name = row['solver_name']
        wall_time = row['wall_time']
        success = row['success']
        speedup = row.get('speedup', 1.0)
        
        status = "✅" if success else "❌"
        print(f"{solver_name:12} {status} Time: {wall_time:6.3f}s  Speedup: {speedup:5.2f}x")

# Timing statistics
if 'timing_stats' in performance_report:
    stats = performance_report['timing_stats']
    print(f"\nTiming Statistics:")
    print(f"- Mean time: {stats['mean_wall_time']:.3f} ± {stats['std_wall_time']:.3f}s")
    print(f"- Range: {stats['min_wall_time']:.3f} - {stats['max_wall_time']:.3f}s")

# Performance vs Accuracy tradeoff visualization
plt.figure(figsize=(10, 6))

methods_for_plot = []
times_for_plot = []
errors_for_plot = []

for method, errors in comparison_results.data['dipole_errors'].items():
    if method in [r['solver_name'].lower() for r in perf_analyzer.benchmark_results]:
        # Find corresponding performance data
        for perf_data in perf_analyzer.benchmark_results:
            if perf_data['solver_name'].lower() == method and perf_data['success']:
                methods_for_plot.append(method)
                times_for_plot.append(perf_data['wall_time'])
                errors_for_plot.append(errors['mae'])
                break

if times_for_plot:
    plt.scatter(times_for_plot, errors_for_plot, s=100, alpha=0.7)
    for i, method in enumerate(methods_for_plot):
        plt.annotate(method, (times_for_plot[i], errors_for_plot[i]), 
                    xytext=(5, 5), textcoords='offset points')

    plt.xlabel('Computation Time (s)')
    plt.ylabel('Mean Absolute Error (vs Exact)')
    plt.title('Performance vs Accuracy Tradeoff')
    plt.yscale('log')
    plt.grid(True, alpha=0.3)
    plt.show()

print("✅ Performance analysis completed successfully!")


In [ ]:
# Step 5: Data Export and Summary

print("\n=== Step 5: Data Export ===")

# Export comparison results to different formats
print("Exporting analysis results...")

try:
    # Export to CSV
    DataExporter.export_comparison_results(
        comparison_results, 
        'analysis_comparison_results.csv', 
        format='csv'
    )
    print("✅ Exported comparison results to CSV")

    # Export time series data
    dipole_export_data = {
        'time': comparison_results.data['times'],
        **{f'dipole_{method}': vals for method, vals in comparison_results.data['dipole_moments'].items()}
    }
    DataExporter.export_time_series_csv(dipole_export_data, 'dipole_time_series.csv')
    print("✅ Exported dipole time series to CSV")

    # Export spectra data
    if spectra_results:
        spectra_export_data = {'energy_eV': list(spectra_results.values())[0]['omega_ev']}
        for method, spectrum_data in spectra_results.items():
            spectra_export_data[f'spectrum_{method}'] = spectrum_data['absorption_spectrum']
        
        DataExporter.export_time_series_csv(spectra_export_data, 'absorption_spectra.csv', time_column='energy_eV')
        print("✅ Exported absorption spectra to CSV")

    # Export performance data
    perf_df.to_csv('performance_benchmarks.csv', index=False)
    print("✅ Exported performance benchmarks to CSV")

except Exception as e:
    print(f"❌ Export failed: {e}")

# Analysis Summary Report
print("\n" + "="*60)
print("               ANALYSIS SUMMARY REPORT")
print("="*60)

print(f"\n📊 SYSTEM CONFIGURATION:")
print(f"   • Molecule: {analysis_config.atom_string}")
print(f"   • Basis set: {analysis_config.basis}")
print(f"   • Qubits: {system.static_hamiltonian.num_qubits}")
print(f"   • Ground energy: {ground_energy:.6f} hartree")

print(f"\n⏱️  PERFORMANCE RESULTS:")
successful_methods = [method for method, data in results_dict.items() if data['success']]
print(f"   • Methods tested: {len(results_dict)}")
print(f"   • Successful runs: {len(successful_methods)}")
if perf_df is not None and not perf_df.empty:
    fastest_method = perf_df.loc[perf_df['success'] & (perf_df['wall_time'] == perf_df[perf_df['success']]['wall_time'].min())]['solver_name'].iloc[0]
    fastest_time = perf_df[perf_df['success']]['wall_time'].min()
    print(f"   • Fastest method: {fastest_method} ({fastest_time:.3f}s)")

print(f"\n🎯 ACCURACY RESULTS:")
if comparison_results.data['dipole_errors']:
    best_method = min(comparison_results.data['dipole_errors'].items(), key=lambda x: x[1]['mae'])
    print(f"   • Most accurate: {best_method[0]} (MAE: {best_method[1]['mae']:.2e})")
    
    avg_fidelities = {}
    for method, fid_array in comparison_results.data['fidelities'].items():
        if method != 'exact':
            avg_fidelities[method] = np.mean(fid_array[~np.isnan(fid_array)])
    
    if avg_fidelities:
        highest_fid_method = max(avg_fidelities.items(), key=lambda x: x[1])
        print(f"   • Highest fidelity: {highest_fid_method[0]} ({highest_fid_method[1]:.6f})")

print(f"\n📈 SPECTRUM ANALYSIS:")
if spectra_results:
    print(f"   • Methods analyzed: {len(spectra_results)}")
    for method, spectrum_data in spectra_results.items():
        omega_ev = spectrum_data['omega_ev']
        sigma = spectrum_data['absorption_spectrum']
        peak_idx = np.argmax(sigma)
        peak_energy = omega_ev[peak_idx]
        print(f"   • {method}: Peak at {peak_energy:.3f} eV")

print(f"\n💾 DATA EXPORT:")
print(f"   • Comparison results: analysis_comparison_results.csv")
print(f"   • Dipole time series: dipole_time_series.csv")
print(f"   • Absorption spectra: absorption_spectra.csv")
print(f"   • Performance data: performance_benchmarks.csv")

print("\n" + "="*60)
print("🎉 ANALYSIS DEMO COMPLETED SUCCESSFULLY!")
print("📊 The refactored analysis module provides comprehensive")
print("   tools for quantum simulation analysis and comparison!")
print("="*60)
